# An Example Jupyter Notebook using MiniZinc
### Peter Denno 2018-09-20

The goal of this notebook is only (1) to get you started with notebooks, and (2) let you see how MiniZinc can be interleaved with Python code.

The following are some instructions for installing Python, Jupyter, and Minizinc. I suppose that the first time you are reading this, you are looking at this file in the git repository. Once you have performed the 7 steps below, you can read *and execute* this file in a Jupyter notebook. By the way, if these steps sound too arduous for students, check out [Jupyterhub](https://jupyterhub.readthedocs.io/en/stable/). Jupyterhub is the best way to run Jupyter in industrial settings too. Note also that as of this writing I've only tried this on a mac. 

1. If Python isn't installed, install it now using whatever method seems best for your situation. Google for it. I am using Python 3.7. You can see what python you are running by running `python --version` in a shell/cmd.exe.
2. Pip or pip3 should have been installed as part of the python install. Run `pip3 install jupyter` to install jupyter.
3. Install Minizinc from [here](http://www.minizinc.org/ide/).
4. Run `pip3 install iminizinc` to install the module for minizinc magic. In case somethign goes wroing, discussion on that installation can be found [here](http://www.minizinc.org/doc-2.2.0/en/jupyter.html).
5. Install git if you don't have it all ready. (Google for instructions.)
6. Clone this respoitory (so that you have this file locally). `git clone https://github.com/pdenno/minizinc-notebooks.git` 
7. You are now ready to run this notebook! In a shell/cmd.exe change to the minizinc-notebooks directory and run `jupyter notebook`. If all goes well, the system will respond with something like this:

```
Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=98b918b7eec119d8d626a2bab4c3e718d440666383597dd9`
```
The copy/pasting is only necessary in this quick and dirty demonstration. If you do it, the  browser page that you get should look like this: 

![alt text](https://raw.githubusercontent.com/pdenno/minizinc-notebooks/master/images/jupyter-files-view.png "Logo Title Text 1")


## Running Cells

Now that you are in your own notebook (You did the stuff above and you are now reading this in your own executing copy, right?), you can run the notebook cell by cell by using the "Run" button above or hitting Shift-Return. In this example, we aren't interested in what is happening in the cells, we are just trying to learn the basics of notebooks. 

Shift-Return on the next cell loads the minizinc magic. More Shift-Returns execute subsequent cells. Give it a try.

In [63]:
 %load_ext iminizinc  

The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


### nqueens Problem

nqueens is the problem of placing n queens on a nxn chess board such that they can't attack each other in one move. You can read more about it [here](https://en.wikipedia.org/wiki/Eight_queens_puzzle). There probably isn't an industrial problem quite like nqueens, but it is an easy problem to describe, so we start with that.

In the following cells, I could have used `int: n = 8;` inside the MiniZinc code below, but it makes much more sense to use a python variable for that. (Imagine that you have a lot of data you want to get into MiniZinc; this is how you would do it.) 

In [64]:
n = 8

We've only written one line of python so far (!) but let's drop in some MiniZinc code. Cells containing whole MiniZinc programs begin with `%%minizinc` "magic."

Here is the nqueens problem from the page on iminizinc mentioned above.

In [65]:
%%minizinc

        include "globals.mzn";
        int: n;
        array[1..n] of var 1..n: queens;        
        constraint all_different(queens);
        constraint all_different([queens[i]+i | i in 1..n]);
        constraint all_different([queens[i]-i | i in 1..n]);
        solve satisfy;

{'queens': [4, 2, 7, 3, 6, 8, 5, 1]}

### The Linear Assignment Problem

The linear assignment problem is the problem of assigning n workers n job most cost effectively. Each worker has a cost incurred in doing each of the jobs.

You can read a little bit about the assignment problem and many other classic combinatorial problems [here](https://glossary.informs.org/ver2/mpgwiki/index.php?title=Category:Common_Problems).

We'll start by defining a nested Python list of the costs. The first element of the list are the costs for the three jobs for the first worker, etc..

In [66]:
cost = [[10, 20, 13], [22, 11, 31], [14, 20, 18]]

Notice that the MiniZinc code below uses a variable n, which is the number of jobs and workers. It makes sense to set that as a python variable. It would make sense to use a python variable for that too, but there is a bug or limitation in the minizinc magic, so we'll leave it in the minizinc for now.

In [67]:
%%minizinc -m bind
include "alldifferent.mzn";
% Each worker has a cost for doing each task. Every worker does one task.
% This is the linear assignment problem -- # of tasks = # of workers.
int: n = 3;
set of int: Workers = 1..n;
set of int: Tasks   = 1..n;
array[Workers, Tasks] of int: cost;

array[Workers] of var Tasks: doesTask; 

constraint alldifferent(doesTask); % implies all tasks get done.

solve minimize sum (w in Workers) (cost[w,doesTask[w]]);

Notice in the minizinc magic line (first line in cell above) we used `-m bind`. That causes the decision variables to be bound to a python variable of the same name. There is an array of three decision variables above defined by `array[Workers] of var Tasks: doesTask`. I promised not to try to teach MiniZinc in this example, so I'll only say that `doesTask` is the array of three variables indexed by `1..n` where n is the python variable set to 3.

It is useful to have such variables in python when you want, for example, to turn the answer that MiniZinc provides into something useful. For example, you could serve a web page telling the workers what job they were assigned.

In [68]:
doesTask

[3, 2, 1]

That's it for now. Have fun with this notebook, MiniZinc and Jupyter notebooks! A nice thing to try would be to start a new notebook, copy and paste the assignment problem and other necessary pieces of this one and see if you could change the code so that it uses an inverse of "doesTask" (perhaps called "whoDoes" -- indexed by jobs rather than workers). Your intution is probably correct. Or if that sounds like too much, just change the cost data to get different job assignments. A 77-page tutorial for MiniZinc can be found [here](http://www.minizinc.org/downloads/doc-latest/minizinc-tute.pdf).